In [10]:
from shapely.geometry import LineString, Point, MultiPoint, Polygon

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import geopandas as gpd
import numpy as np
from datetime import datetime
from shapely.geometry import LineString, Point, MultiPoint, Polygon
from IPython.display import display, clear_output


path_to_data = gpd.datasets.get_path("nybb")
gdf = gpd.read_file(path_to_data)
manh = gdf.loc[gdf.BoroName=='Manhattan'].to_crs(4326)

roads = gpd.read_file('data/nyc_roads/geo_export_05dcab6d-50ed-427f-aae3-61ecc2069210.shp', mask=manh)[['trafdir', 'geometry']]

In [11]:
notw = roads[roads.trafdir!='TW']

In [12]:
tw = roads[roads.trafdir=='TW']

In [14]:
notw = roads[roads.trafdir!='TW']
tw = roads[roads.trafdir=='TW']
tw['trafdir'] = 'TF'
tw2 = tw.copy()
tw2['trafdir'] = 'FT'
roadsv2 = pd.concat([notw, tw, tw2])
roadsv2 = roadsv2.loc[roadsv2.trafdir!='NV'].reset_index(drop=True)
sps = []
eps = []
for _,road in roadsv2.iterrows():
    if road.trafdir == 'FT':
        sps.append(road.geometry.boundary[0]) 
        eps.append(road.geometry.boundary[1]) 

    elif road.trafdir == 'TF':
        sps.append(road.geometry.boundary[1]) 
        eps.append(road.geometry.boundary[0]) 

In [22]:
def meanpoints(lst, default):
    if len(lst)==0:
        print('same')
        return default
    else:
        return Point(np.mean([[a.x, a.y] for a in lst], axis=0))

roadsv2 = pd.concat([notw, tw, tw2])
roadsv2 = roadsv2.loc[roadsv2.trafdir!='NV'].reset_index(drop=True)
sps = []
eps = []
for _,road in roadsv2.iterrows():
    if road.trafdir == 'FT':
        sps.append(road.geometry.boundary[0]) 
        eps.append(road.geometry.boundary[1]) 

    elif road.trafdir == 'TF':
        sps.append(road.geometry.boundary[1]) 
        eps.append(road.geometry.boundary[0]) 

starts = gpd.GeoDataFrame(geometry=roadsv2.start, crs=4326)
starts['label'] = 'start'
stops = gpd.GeoDataFrame(geometry=roadsv2.end, crs=4326)
stops['label'] = 'end'

sall = pd.concat([starts, stops])
X = sall.geometry.apply(lambda a: np.array([a.x, a.y]))
X = np.vstack([val for val in X.values])

from sklearn.cluster import DBSCAN
import numpy as np


clustering = DBSCAN(eps=1e-4, min_samples=2).fit(X)
sall['cls'] = clustering.labels_

pnts = gpd.GeoSeries(sall.groupby('cls').apply(lambda a: meanpoints(a.geometry.tolist(),0))[1:], crs=4326)

for i in tqdm(range(len(roadsv2))):
    start_p = sall.loc[i].iloc[0].cls
    end_p = sall.loc[i].iloc[1].cls
    # break
    roadsv2.loc[i, 'start2'] = start_p
    roadsv2.loc[i, 'end2'] = end_p

rds = roadsv2.loc[(roadsv2.start2!=-1) & (roadsv2.end2!=-1)]
rds['weights'] = rds.apply(lambda a: pnts[a.start2].distance(pnts[a.end2]), axis=1)


rds = rds.drop(['geometry'], axis=1)
rds.rename({'start2':'start_id', 'end2':'end_id'}, axis=1, inplace=True)
rds.to_pickle('roadsfornx.pkl')
pnts.to_pickle('reps.pkl')

In [89]:
starts = gpd.GeoDataFrame(geometry=roadsv2.start, crs=4326)
starts['label'] = 'start'
stops = gpd.GeoDataFrame(geometry=roadsv2.end, crs=4326)
stops['label'] = 'end'

In [193]:
sall = pd.concat([starts, stops])


In [202]:
X = sall.geometry.apply(lambda a: np.array([a.x, a.y]))

In [207]:
X = np.vstack([val for val in X.values])

In [185]:

def meanpoints(lst, default):
    if len(lst)==0:
        print('same')
        return default
    else:
        return Point(np.mean([[a.x, a.y] for a in lst], axis=0))

In [214]:
from sklearn.cluster import DBSCAN
import numpy as np
# X = np.array([[1, 2], [2, 2], [2, 3],
#               [8, 7], [8, 8], [25, 80]])
clustering = DBSCAN(eps=1e-4, min_samples=2).fit(X)
sall['cls'] = clustering.labels_

In [223]:
pnts = gpd.GeoSeries(sall.groupby('cls').apply(lambda a: meanpoints(a.geometry.tolist(),0))[1:], crs=4326)

In [248]:
for i in tqdm(range(len(roadsv2))):
    start_p = sall.loc[i].iloc[0].cls
    end_p = sall.loc[i].iloc[1].cls
    # break
    roadsv2.loc[i, 'start2'] = start_p
    roadsv2.loc[i, 'end2'] = end_p

  0%|          | 0/13665 [00:00<?, ?it/s]

In [307]:
rds = roadsv2.loc[(roadsv2.start2!=-1) & (roadsv2.end2!=-1)]

In [309]:
rds['weights'] = rds.apply(lambda a: pnts[a.start2].distance(pnts[a.end2]), axis=1)

In [315]:
rds.weights.max()

0.020539932900730366

In [323]:
rds = rds.drop(['geometry'], axis=1)
rds.rename({'start2':'start_id', 'end2':'end_id'}, axis=1, inplace=True)
rds.to_pickle('roadsfornx.pkl')
pnts.to_pickle('reps.pkl')

In [325]:
rds.rename({'start2':'start_id', 'end2':'end_id'}, axis=1, inplace=True)

In [327]:
rds.to_pickle('roadsfornx.pkl')

In [317]:
import networkx as nx

G = nx.DiGraph()


In [329]:
pnts.to_pickle('reps.pkl')

In [318]:
G.add_nodes_from(range(len(pnts)))

In [319]:
for a,b,c in tqdm(rds[['start2','end2', 'weights']].values):
    G.add_edge(a, b, weight=c)
# G.remove_node(-1)

  0%|          | 0/13630 [00:00<?, ?it/s]

In [299]:
roadsv2.loc[roadsv2.start2==-1]

,trafdir,geometry,start,end,connections,start2,end2
226,TF,POINT (-73.92760 40.78104),POINT (-73.92565 40.77843),POINT (-73.92760 40.78104),"[670, 1679]",-1,639
441,FT,POINT (-73.92842 40.84708),POINT (-73.92728 40.84655),POINT (-73.92842 40.84708),"[1505, 1689, 3693, 4818, 6089]",-1,1373
515,TF,POINT (-73.99913 40.70798),POINT (-73.99457 40.70438),POINT (-73.99913 40.70798),[8697],-1,5219
1113,TF,POINT (-74.00843 40.68602),POINT (-74.00644 40.68343),POINT (-74.00843 40.68602),[1079],-1,1008
1451,TF,POINT (-73.99177 40.70956),POINT (-73.98931 40.70515),POINT (-73.99177 40.70956),"[38, 1082, 2798, 5629, 6865]",-1,38
2599,TF,POINT (-73.94129 40.76966),POINT (-73.94087 40.76994),POINT (-73.94129 40.76966),"[5462, 9627]",-1,3954
2805,TF,POINT (-73.96812 40.74773),POINT (-73.96118 40.74301),POINT (-73.96812 40.74773),"[512, 2709]",-1,491
3116,TF,POINT (-74.01161 40.72607),POINT (-74.02151 40.72749),POINT (-74.01161 40.72607),[134],-1,134
3442,FT,POINT (-74.01568 40.70126),POINT (-74.01448 40.69780),POINT (-74.01568 40.70126),[8421],-1,5114
3527,FT,POINT (-73.97666 40.77674),POINT (-73.97627 40.77724),POINT (-73.97666 40.77674),[11464],-1,6042


In [266]:
!conda install -c conda-forge networkx

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
\ 

In [297]:
pnts

cls
0       POINT (-74.00826 40.74467)
1       POINT (-73.96268 40.76072)
2       POINT (-73.98391 40.71130)
3       POINT (-73.94733 40.83089)
4       POINT (-73.93419 40.81409)
                   ...            
6127    POINT (-73.93944 40.81262)
6128    POINT (-74.00032 40.72258)
6129    POINT (-73.95085 40.75520)
6130    POINT (-73.94687 40.85059)
6131    POINT (-73.96891 40.71249)
Length: 6132, dtype: geometry

In [267]:
A = nx.nx_agraph.to_agraph(G)  # convert to a graphviz graph
A.layout()  # neato layout
A.draw("k5.ps") 

ImportError: requires pygraphviz http://pygraphviz.github.io/

In [271]:
print(nx.shortest_path(G, source=0, target=4))

[0, 4429, 3016, 3715, 4794, 3550, 3535, 3671, 3835, 4991, 4914, 2750, 3074, 3284, 2795, 4739, 4628, 4221, 913, -1, 858, 1478, 918, 4066, 2745, 330, 4]


In [330]:
G

In [331]:
for i in range(10):

    st, ed = pnts.distance(startsdf.iloc[i].geometry).argmin(), pnts.distance(stopsdf.iloc[i].geometry).argmin()
    print(nx.shortest_path(G, source=st, target=ed, weight='weight'))
    ls = LineString(pnts.loc[nx.shortest_path(G, source=st, target=ed, weight='weight')].geometry.tolist())
    viz2(ls, startsdf.iloc[i:i+1], stopsdf.iloc[i:i+1])
    input()
    clear_output()


In [290]:
from misc import viz2

In [332]:
nx.write_gpickle(G, "road_graph.gpickle")

In [279]:
df = pd.read_csv('data/train.csv')

startsdf= gpd.GeoDataFrame(df[['id']], geometry=gpd.points_from_xy(df.pickup_longitude, df.pickup_latitude), crs=4326)
stopsdf = gpd.GeoDataFrame(df[['id']], geometry=gpd.points_from_xy(df.dropoff_longitude, df.dropoff_latitude), crs=4326)

path_to_data = gpd.datasets.get_path("nybb")
gdf = gpd.read_file(path_to_data)
manh = gdf.loc[gdf.BoroName=='Manhattan'].to_crs(4326)

# roads = gpd.read_file('data/nyc_roads/geo_export_05dcab6d-50ed-427f-aae3-61ecc2069210.shp', mask=manh)

sind = gpd.tools.sjoin(startsdf, manh, op='within').index.tolist()
eind = gpd.tools.sjoin(stopsdf, manh, op='within').index.tolist()
df = df.loc[list(set(sind).intersection(set(eind)))]
startsdf = gpd.GeoDataFrame(df[['id']], geometry=gpd.points_from_xy(df.pickup_longitude, df.pickup_latitude), crs=4326)
stopsdf = gpd.GeoDataFrame(df[['id']], geometry=gpd.points_from_xy(df.dropoff_longitude, df.dropoff_latitude), crs=4326)